<a href="https://colab.research.google.com/github/Rohith-Rongali/IMP/blob/main/pruning_puzzles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class DNN(nn.Module):
    def __init__(self, dim_in, dim_out, width, depth):
        super(DNN, self).__init__()
        self.depth = depth
        self.layers = nn.ModuleList([nn.Linear(dim_in if i == 0 else width, width) for i in range(self.depth)])
        self.output_layer = nn.Linear(width, dim_out)
        self.relu = nn.ReLU()

    def forward(self, x):
        for layer in self.layers:
            x = self.relu(layer(x))
        x = self.output_layer(x)
        return x
model = DNN(4,2,10,3).to(device)

In [ ]:
list(model.layers[0].named_parameters())

[('weight',
  Parameter containing:
  tensor([[ 0.1877,  0.3423, -0.4911,  0.3108],
          [-0.4713,  0.1970, -0.0298,  0.1402],
          [ 0.1899, -0.4023,  0.1890,  0.4711],
          [ 0.1416,  0.3241,  0.3229,  0.3026],
          [ 0.0932,  0.4593, -0.2672,  0.0754],
          [-0.0176, -0.0359, -0.3718,  0.3059],
          [-0.1632, -0.0459, -0.3760,  0.4926],
          [-0.2628,  0.3024,  0.2096, -0.0395],
          [-0.2094, -0.2710, -0.3218,  0.4083],
          [ 0.3479, -0.2455, -0.3812, -0.1856]], requires_grad=True)),
 ('bias',
  Parameter containing:
  tensor([-0.1329,  0.4386,  0.2655, -0.4709, -0.1128,  0.3792,  0.3811,  0.3990,
          -0.1714,  0.2147], requires_grad=True))]

In [ ]:
list(model.layers[0].named_buffers())

[]

In [ ]:
prune.random_unstructured(model.layers[0], name="weight", amount=0.3)

Linear(in_features=4, out_features=10, bias=True)

In [ ]:
list(model.layers[0].named_parameters())

[('bias',
  Parameter containing:
  tensor([-0.1329,  0.4386,  0.2655, -0.4709, -0.1128,  0.3792,  0.3811,  0.3990,
          -0.1714,  0.2147], requires_grad=True)),
 ('weight_orig',
  Parameter containing:
  tensor([[ 0.1877,  0.3423, -0.4911,  0.3108],
          [-0.4713,  0.1970, -0.0298,  0.1402],
          [ 0.1899, -0.4023,  0.1890,  0.4711],
          [ 0.1416,  0.3241,  0.3229,  0.3026],
          [ 0.0932,  0.4593, -0.2672,  0.0754],
          [-0.0176, -0.0359, -0.3718,  0.3059],
          [-0.1632, -0.0459, -0.3760,  0.4926],
          [-0.2628,  0.3024,  0.2096, -0.0395],
          [-0.2094, -0.2710, -0.3218,  0.4083],
          [ 0.3479, -0.2455, -0.3812, -0.1856]], requires_grad=True))]

In [ ]:
list(model.layers[0].named_buffers())

[('weight_mask',
  tensor([[1., 1., 1., 1.],
          [0., 1., 1., 1.],
          [1., 0., 0., 1.],
          [0., 1., 0., 0.],
          [1., 0., 1., 0.],
          [1., 1., 1., 1.],
          [1., 0., 1., 1.],
          [1., 0., 1., 1.],
          [1., 1., 1., 0.],
          [1., 0., 1., 1.]]))]

In [ ]:
model.layers[0].weight

tensor([[ 0.1877,  0.3423, -0.4911,  0.3108],
        [-0.0000,  0.1970, -0.0298,  0.1402],
        [ 0.1899, -0.0000,  0.0000,  0.4711],
        [ 0.0000,  0.3241,  0.0000,  0.0000],
        [ 0.0932,  0.0000, -0.2672,  0.0000],
        [-0.0176, -0.0359, -0.3718,  0.3059],
        [-0.1632, -0.0000, -0.3760,  0.4926],
        [-0.2628,  0.0000,  0.2096, -0.0395],
        [-0.2094, -0.2710, -0.3218,  0.0000],
        [ 0.3479, -0.0000, -0.3812, -0.1856]], grad_fn=<MulBackward0>)

In [ ]:
prune.ln_structured(model.layers[0], name="weight", amount=0.5, n=2, dim=0)

Linear(in_features=4, out_features=10, bias=True)

In [ ]:
list(model.layers[0].named_buffers())

[('weight_mask',
  tensor([[1., 1., 1., 1.],
          [0., 0., 0., 0.],
          [1., 0., 0., 1.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [1., 1., 1., 1.],
          [1., 0., 1., 1.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [1., 0., 1., 1.]]))]

In [ ]:
for hook in model.layers[0]._forward_pre_hooks.values():
    if hook._tensor_name == "weight":  # select out the correct hook
        break

print(list(hook))

[<torch.nn.utils.prune.RandomUnstructured object at 0x78f836e9e590>, <torch.nn.utils.prune.LnStructured object at 0x78f836e9e5f0>]


In [20]:
model.output_layer.weight

Parameter containing:
tensor([[ 0.1902, -0.2616, -0.3089, -0.0460,  0.0114, -0.0508,  0.1597,  0.2594,
         -0.2004, -0.0994],
        [ 0.0197, -0.2390,  0.1228, -0.0423,  0.0973,  0.2530, -0.2046, -0.1634,
          0.2472,  0.0101]], requires_grad=True)

In [21]:
prune.l1_unstructured(model.output_layer, name="weight", amount=0.1)

Linear(in_features=10, out_features=2, bias=True)

In [24]:
list(model.output_layer.named_parameters())

[('bias',
  Parameter containing:
  tensor([-0.0364, -0.0301], requires_grad=True)),
 ('weight_orig',
  Parameter containing:
  tensor([[ 0.1902, -0.2616, -0.3089, -0.0460,  0.0114, -0.0508,  0.1597,  0.2594,
           -0.2004, -0.0994],
          [ 0.0197, -0.2390,  0.1228, -0.0423,  0.0973,  0.2530, -0.2046, -0.1634,
            0.2472,  0.0101]], requires_grad=True))]